In [17]:
#imports
import pandas as pd 
import numpy as np

In [58]:
# Load the Excel file
file_path = "DataLab2.xlsx" 

# Read the first 501 to 1009 rows and first 11 columns
df = pd.read_excel(file_path, header=0, skiprows=range(1, 501), usecols=range(11))

# Transform 'PL' column to create 'Losses' column
df["Losses"] = df["PL"] * (-1)

# extract the years
df["Year"] = df["Date"].dt.year.astype(int)

# Display the first few rows (for testing)
df.head()


,Date,PL,VaRBHS,VaREWMA,VaRn,VaRt,VaRPot,ESEWMA,ESn,ESt,ESPot,Losses,Year
0,2006-12-26,-150,1115.0,1147.464693,777.535296,986.872911,1391.726372,2544.116901,897.215494,3209.017239,1924.333523,150,2006
1,2006-12-28,150,1115.0,1144.195603,778.321836,987.104816,1391.726372,2540.137912,898.017553,3189.914672,1924.333523,-150,2006
2,2006-12-29,-20,1115.0,1117.460475,778.334975,987.885905,1391.726372,2506.631070,898.020953,3203.255498,1924.333523,20,2006
3,2006-12-31,30,1115.0,1085.901247,777.503184,989.976454,1391.726372,2468.406961,897.126264,3261.771712,1924.333523,-30,2006
4,2007-01-01,30,1115.0,1053.756229,777.645897,990.797723,1391.726372,2431.278879,897.263546,3270.873551,1924.333523,-30,2007


In [126]:
# Backtesting VAR

# encode exceptions for each VAR
var_indices = list(range(2,7)) # these are the columns indices for the VAR

# to get all the names of the headers for the VAR so we can use these when encoding
var_names = df.columns[var_indices] 

# here we create the headers for the exceptions, 0 no exception, 1 is.
ex_names = [f'{var_name}_exception' for var_name in var_names] 

# we zip the lists just to be able to loop over both
var_ex_names = list(zip(var_names, ex_names)) 

# here we encode a violation (exception) as a 1, otherwise 0
for var_name, ex_name in var_ex_names:
    df[ex_name] = (df[var_name] < df["Losses"]).astype(int) 

# Get indices of the exception columns (for easy retrieval later)
ex_indices = [df.columns.get_loc(ex_name) for ex_name in ex_names]


In [144]:
# now to actually doing the tests for each year, alpha=0.99 was used in the first exercise for the vars, so we use that here
# first create a python dictionary for all the years, so we extract each row (with relevant data) for each year

years_data = {}
loss_index = [df.columns.get_loc("Losses")]
var_ex_loss_indices = var_indices + ex_indices + loss_index

for row, year in enumerate(df["Year"]):
    data = df.iloc[row, var_ex_loss_indices]
    if year in years_data:
        years_data[year].append(data)
    else:
        years_data[year] = [data]

# convert each year into a separate dataframe for easy analysis
for year in years_data:
    years_data[year] = pd.DataFrame(years[year])
    
#print(years_data[2006])


# now we have a python dict with each year with the corresponding losses for that year, and the VARs for those years, exceptions in order aswell inside each year, since we append.

In [128]:
# now we will actually be doing the tests on each of the VARs
# since we only really care about underestimating losses, we will do one sided tests

#will be using the binomial distribution from scipy
from scipy.stats import binom


In [172]:
#defining a function for the kupiec test
def kupiec(ex_list: list, alpha=0.99, significance_level=0.05) -> str:
    # Total number of observations (days)
    n = len(ex_list)
    
    # Number of violations (exceptions should be 1 for a violation, 0 otherwise)
    k = sum(ex_list)
    
    # The expected probability of a violation under the null hypothesis
    p = 1 - alpha

    # Handle the case with 0 violations explicitly for clarity
    if k == 0:
        p_value = 1.0
    else:
        # For k >= 1, compute the p-value as the probability of seeing at least k violations
        p_value = 1 - binom.cdf(k - 1, n, p)
    
    # Return True if we reject the null hypothesis (p-value is less than the significance level)
    test = p_value < significance_level 
    if test:
        return "Reject"
    else:
        return "Accept"
    

In [155]:


# function for basel
def basel(ex_list: list, alpha = 0.99) -> str:
    n = len(ex_list)
    p = 1-alpha
    k = sum(ex_list)  # Count exceptions

    # Compute the cutoffs dynamically
    green_yellow = int(binom.ppf(0.95, n, p))  # 95% cutoff
    yellow_red = int(binom.ppf(0.99, n, p))  # 99% cutoff

    # Assign traffic light zone
    if k <= green_yellow:
        return "Green"
    elif green_yellow < k <= yellow_red:
        return "Yellow"
    else:
        return "Red"




In [ ]:
# function for christofferson
from scipy.stats import chi2
def christofferson(violations: list):

    # initialize transition counts
    n_00 = n_01 = n_10 = n_11 = 0

    


    
    pass

In [178]:
# here we will be performing each of the tests

# create a dictionary keeping track of the test results, for each year, on each var (very complicated structure but works wonders)
test_data = {
    year: 
               {
        var_name:
                    {
    "Kupiec":None, 
    "Basel":None,
    "Christofferson":None
                    }
    for var_name in var_names
               }
    for year in years
            }
#print(test_data)

In [179]:
# loop over all the years, extract the exception columns, and perform each test on each encoded exception
for year, year_dataframe in years_data.items():

    #this gives all the exception columns for that year
    exceptions_columns = year_dataframe[ex_names] 

    #var_ex_names is a zipped list with the var and corresponding exceptions column name
    for var_name, ex_name in var_ex_names: 
        
        # the violations for that current type of var for that year
        ex_list = exceptions_columns[ex_name] 

         # here we do the kupiec test and store everything for that test, that var, that year
        test_data[year][var_name]["Kupiec"] = kupiec(ex_list)

        # here we do the basel test
        test_data[year][var_name]["Basel"] = basel(ex_list)

        


In [181]:
print(test_data)

{2006: {'VaRBHS': {'Kupiec': 'Accept', 'Basel': 'Green', 'Christofferson': None}, 'VaREWMA': {'Kupiec': 'Accept', 'Basel': 'Green', 'Christofferson': None}, 'VaRn': {'Kupiec': 'Accept', 'Basel': 'Green', 'Christofferson': None}, 'VaRt': {'Kupiec': 'Accept', 'Basel': 'Green', 'Christofferson': None}, 'VaRPot': {'Kupiec': 'Accept', 'Basel': 'Green', 'Christofferson': None}}, 2007: {'VaRBHS': {'Kupiec': 'Reject', 'Basel': 'Red', 'Christofferson': None}, 'VaREWMA': {'Kupiec': 'Accept', 'Basel': 'Green', 'Christofferson': None}, 'VaRn': {'Kupiec': 'Reject', 'Basel': 'Red', 'Christofferson': None}, 'VaRt': {'Kupiec': 'Reject', 'Basel': 'Red', 'Christofferson': None}, 'VaRPot': {'Kupiec': 'Accept', 'Basel': 'Green', 'Christofferson': None}}, 2008: {'VaRBHS': {'Kupiec': 'Reject', 'Basel': 'Yellow', 'Christofferson': None}, 'VaREWMA': {'Kupiec': 'Reject', 'Basel': 'Yellow', 'Christofferson': None}, 'VaRn': {'Kupiec': 'Reject', 'Basel': 'Red', 'Christofferson': None}, 'VaRt': {'Kupiec': 'Reject'